<a href="https://colab.research.google.com/github/GaohuiXu/LLM-NLP/blob/main/course/zh-CN/chapter2/section2_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 管道的内部 (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [20]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I love my girlfriend.",
        "I want to play games!",
        "I am a bad student."
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998421669006348},
 {'label': 'POSITIVE', 'score': 0.9980376362800598},
 {'label': 'NEGATIVE', 'score': 0.9997832179069519}]

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [19]:
raw_inputs = [
    "I love my girlfriend.",
        "I want to play games!",
        "I am a bad student."
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") # padding=True 自动补齐
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2293, 2026, 6513, 1012,  102,    0],
        [ 101, 1045, 2215, 2000, 2377, 2399,  999,  102],
        [ 101, 1045, 2572, 1037, 2919, 3076, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [23]:
tokens = tokenizer.tokenize("I love my girlfriend.")
ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(ids)

['i', 'love', 'my', 'girlfriend', '.']
[1045, 2293, 2026, 6513, 1012]


In [24]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [25]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([3, 8, 768])


In [26]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [27]:
print(outputs.logits.shape)

torch.Size([3, 2])


In [28]:
print(outputs.logits)

tensor([[-4.2101,  4.5437],
        [-3.0885,  3.1432],
        [ 4.6667, -3.7699]], grad_fn=<AddmmBackward0>)


In [29]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[1.5784e-04, 9.9984e-01],
        [1.9623e-03, 9.9804e-01],
        [9.9978e-01, 2.1675e-04]], grad_fn=<SoftmaxBackward0>)


In [12]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}